In [1]:
import torch
import torch.nn as nn


class TargetModel(nn.Module):
    """
        Target model for classification.
    """
    def __init__(self):
        super(TargetModel, self).__init__()
        self.fc1 = nn.Linear(30, 64)
        self.fc2 = nn.Linear(64, 2)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return torch.softmax(x, dim=1)

In [2]:
class ShadowModel(nn.Module):
    """
        Shadow model for membership inference.
    """
    def __init__(self):
        super(ShadowModel, self).__init__()
        self.fc1 = nn.Linear(30, 64)
        self.fc2 = nn.Linear(64, 2)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return torch.softmax(x, dim=1)

In [3]:
import torch.optim as optim
from sklearn.metrics import accuracy_score


def train_target_model(X_train, y_train, X_test, y_test, num_epochs):
    """
        Trains the target model.

        Args:
            X_train (torch.Tensor): Training data.
            y_train (torch.Tensor): Training labels.
            X_test (torch.Tensor): Test data.
            y_test (torch.Tensor): Test labels.

        Returns:
            TargetModel: Trained target model.
    """
    target_model = TargetModel()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(target_model.parameters(), lr=0.001)

    for epoch in range(num_epochs):
        optimizer.zero_grad()
        outputs = target_model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()

    with torch.no_grad():
        target_model.eval()
        outputs = target_model(X_test)
        _, predicted = torch.max(outputs, 1)
        accuracy = accuracy_score(predicted.numpy(), y_test.numpy())
        print(f"Accuracy of the target model: {accuracy}")

    return target_model

In [4]:
class Generator(nn.Module):
    """
        Generator network for creating synthetic data.
    """
    def __init__(self, input_size, output_size):
        super(Generator, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

In [5]:
class Discriminator(nn.Module):
    """
        Discriminator network for discriminating
        between real and synthetic data.
    """
    def __init__(self, input_size):
        super(Discriminator, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

In [6]:
def train_gan_single_data_point(data_point, generator, discriminator,
                                num_epochs):
    """
        Trains a GAN using just a single data point.

        Args:
            data_point (numpy.ndarray): Single data point to use for training.
            generator (Generator): Generator network.
            discriminator (Discriminator): Discriminator network.
            num_epochs (int): Number of epochs for training.
    """
    criterion = nn.BCELoss()
    d_optimizer = optim.Adam(discriminator.parameters(), lr=0.001)
    g_optimizer = optim.Adam(generator.parameters(), lr=0.001)

    for epoch in range(num_epochs):
        discriminator.zero_grad()

        real_data = torch.tensor(data_point).float().unsqueeze(0)
        real_labels = torch.ones((1, 1))

        real_output = discriminator(real_data)
        d_real_loss = criterion(real_output, real_labels)

        noise = torch.randn(1, 100)
        fake_data = generator(noise)
        fake_labels = torch.zeros((1, 1))

        fake_output = discriminator(fake_data)
        d_fake_loss = criterion(fake_output, fake_labels)

        d_loss = d_real_loss + d_fake_loss
        d_loss.backward()
        d_optimizer.step()

        generator.zero_grad()
        noise = torch.randn(1, 100)
        fake_data = generator(noise)
        fake_labels = torch.ones((1, 1))

        output = discriminator(fake_data)
        g_loss = criterion(output, fake_labels)

        g_loss.backward()
        g_optimizer.step()

        print(f"Epoch [{epoch+1}/{num_epochs}], Generator Loss: " + \
              f"{g_loss.item():.4f}, Discriminator Loss: {d_loss.item():.4f}")

In [7]:
def train_gan_small_subset(data_subset, generator, discriminator, num_epochs, batch_size=32):
    """
        Trains a GAN using a subset of a dataset.

        Args:
            data_subset (numpy.ndarray): Subset of data points for training.
            generator (Generator): Generator network.
            discriminator (Discriminator): Discriminator network.
            num_epochs (int): Number of epochs for training.
            batch_size (int, optional): Batch size for training. Defaults to 32.
    """
    criterion = nn.BCELoss()
    d_optimizer = optim.Adam(discriminator.parameters(), lr=0.001)
    g_optimizer = optim.Adam(generator.parameters(), lr=0.001)

    for epoch in range(num_epochs):
        for i in range(0, len(data_subset), batch_size):
            # Train Discriminator
            discriminator.zero_grad()

            real_data = torch.tensor(data_subset[i:i+batch_size]).float()
            real_labels = torch.ones((len(real_data), 1))

            real_output = discriminator(real_data)
            d_real_loss = criterion(real_output, real_labels)

            noise = torch.randn(len(real_data), 100)
            fake_data = generator(noise)
            fake_labels = torch.zeros((len(fake_data), 1))

            fake_output = discriminator(fake_data)
            d_fake_loss = criterion(fake_output, fake_labels)

            d_loss = d_real_loss + d_fake_loss
            d_loss.backward()
            d_optimizer.step()

            # Train Generator
            generator.zero_grad()
            noise = torch.randn(len(real_data), 100)
            fake_data = generator(noise)
            fake_labels = torch.ones((len(fake_data), 1))

            output = discriminator(fake_data)
            g_loss = criterion(output, fake_labels)

            g_loss.backward()
            g_optimizer.step()

        print(f"Epoch [{epoch+1}/{num_epochs}], Generator Loss: " + \
              f"{g_loss.item():.4f}, Discriminator Loss: {d_loss.item():.4f}")

In [8]:
import numpy as np


def generate_synthetic_data(generator, num_samples):
    """
        Generates synthetic data using the trained Generator

        Args:
        generator (Generator): Trained Generator network.
        num_samples (int): Number of synthetic samples to generate.

        Returns:
            numpy.ndarray: Synthetic data.
            numpy.ndarray: Synthetic labels.
    """
    noise = torch.randn(num_samples, 100)
    synthetic_data = generator(noise).detach().numpy()
    synthetic_labels = np.random.randint(0, 2, size=(num_samples,))
    return synthetic_data, synthetic_labels

In [9]:
def train_shadow_model(X_train, y_train, X_test, y_test, num_epochs):
    """
        Trains the shadow model.

        Args:
            X_train (torch.Tensor): Training data.
            y_train (torch.Tensor): Training labels.
            X_test (torch.Tensor): Test data.
            y_test (torch.Tensor): Test labels.

        Returns:
            ShadowModel: Trained shadow model.
    """
    shadow_model = ShadowModel()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(shadow_model.parameters(), lr=0.001)

    for epoch in range(num_epochs):
        optimizer.zero_grad()
        outputs = shadow_model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()

    with torch.no_grad():
        shadow_model.eval()
        outputs = shadow_model(X_test)
        _, predicted = torch.max(outputs, 1)
        accuracy = accuracy_score(predicted.numpy(), y_test.numpy())
        print(f"Accuracy of the shadow model: {accuracy}")

    return shadow_model

In [10]:
def membership_inference_attack(original_data, original_labels, shadow_model):
    """
        Performs membership inference attack on the entire original dataset to
        measure the shadow model's accuracy.

        Args:
            original_data (numpy.ndarray): Original data.
            original_labels (numpy.ndarray): Original labels.
            shadow_model (ShadowModel): Shadow model used for inference attack.
    """
    correct_predictions = 0
    total_samples = len(original_data)

    for i in range(total_samples):
        data_point = original_data[i]
        label = original_labels[i]

        # Use the shadow model to predict whether
        # the data point was used in training
        data_point_tensor = torch.tensor(data_point).float().unsqueeze(0)
        with torch.no_grad():
            shadow_model.eval()
            output = shadow_model(data_point_tensor)
            predicted_label = torch.argmax(output).item()

        if label == predicted_label:
            correct_predictions += 1

    attack_accuracy = correct_predictions / total_samples
    print("Membership Inference Attack Accuracy on Original Dataset: " + \
          f"{attack_accuracy}")

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

num_epochs = 100

data = load_breast_cancer()
X = data.data
y = data.target

# Select a single row from the original dataset for GAN training
selected_row_idx = np.random.randint(0, len(X))
selected_data_point = X[selected_row_idx]

# Train the GAN with the selected data point
generator_single_data_point = Generator(100, 30)
discriminator_single_data_point = Discriminator(30)
train_gan_single_data_point(selected_data_point, generator_single_data_point,
                            discriminator_single_data_point, num_epochs)

# Train the GAN with a small subset of the original dataset
selected_subset_indices = np.random.choice(len(X), size=10, replace=False)
selected_subset = X[selected_subset_indices]
generator_subset = Generator(100, 30)
discriminator_subset = Discriminator(30)
train_gan_small_subset(selected_subset, generator_subset, discriminator_subset,
                       num_epochs)

# Generate synthetic data using the Generator trained with single data point
synthetic_data_single_data_point, synthetic_labels_single_data_point = \
    generate_synthetic_data(generator_single_data_point, len(X))

# Generate synthetic data using the Generator trained with the subset of the original data
synthetic_data_subset, synthetic_labels_subset = generate_synthetic_data(
    generator_subset, len(X))

# Split the data into train and test sets for target model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)#, random_state=42)

# Train the target model with entire training dataset
target_model = train_target_model(torch.tensor(X_train).float(),
                                  torch.tensor(y_train),
                                  torch.tensor(X_test).float(),
                                  torch.tensor(y_test), num_epochs)

print("-" * 50)

print("Training shadow model with synthetic dataset created from a single data point...")
shadow_model_single_data_point = train_shadow_model(
    torch.tensor(synthetic_data_single_data_point).float(),
    torch.tensor(synthetic_labels_single_data_point),
    torch.tensor(X_test).float(), torch.tensor(y_test), num_epochs)

print("-" * 50)

print("Training shadow model with synthetic dataset created from a subset of the original dataset...")
shadow_model_subset = train_shadow_model(
    torch.tensor(synthetic_data_subset).float(),
    torch.tensor(synthetic_labels_subset), torch.tensor(X_test).float(),
    torch.tensor(y_test), num_epochs)

print("-" * 50)

print("Running MIA using the shadow model trained on synthetic data created from a single data point...")
membership_inference_attack(X, y, shadow_model_single_data_point)

print("-" * 50)

print("Running MIA using the shadow model trained on synthetic data created from a subset of the original dataset...")
membership_inference_attack(X, y, shadow_model_subset)